In [1]:
import pandas as pd
import spacy
import csv
import re

In [2]:
with open('/Users/camilleko/NLP_summarization/data/interim/summary_ids.txt', 'r') as file:
    lines = file.readlines()

In [3]:
matching_ids = [line.strip() for line in lines]

matching_ids
print(len(matching_ids))

6495


In [4]:
open_df = pd.read_csv('/Users/camilleko/NLP_summarization/data/interim/df_open_ready4EDA.csv',index_col=0)
open_df

,names,ids,audiences,fees,venues,comments,services,taglines
0,RSL Ardrossan Sub Branch,193932,Serving and ex-service men and women and their...,NaN,Hall/Clubrooms for hire,NaN,Welfare and pensions support for ex-servicemen...,NaN
7,"RSL Brinkworth, Koolunga & Yacka Sub Branch",193940,Serving and ex-service men and women and their...,NaN,NaN,We welcome former and serving Defence members ...,Welfare and pensions support for ex-servicemen...,NaN
11,RSL Coonawarra-Penola Sub Branch,193945,Serving and ex-service men and women and their...,NaN,NaN,NaN,Welfare and pensions support for ex-servicemen...,NaN
36,RSL Port Pirie & Military Museum,193975,Serving and ex-service men and women and their...,NaN,Hall/Clubroom,NaN,Welfare and pensions support for ex-servicemen...,NaN
56,Adelaide Turf Cricket Association,194001,NaN,NaN,NaN,NaN,Coordination & promotion of community turf cri...,NaN
...,...,...,...,...,...,...,...,...
14341,Yalata Community Council,238386,NaN,NaN,NaN,NaN,Municipal services\nGeneral Store – Owned and ...,NaN
14342,Gerard Aboriginal Community Council,238387,NaN,NaN,NaN,NaN,Municipal services,NaN
14343,Adelaide Hills Garden Affair,238388,NaN,Tickets available from 1 Dec 2023. All ticket...,NaN,"Stalls, speakers, demonstrations, children's a...",NaN,An event for the love of gardening
14344,Anangu Pitjantjatjara Yankunytjatjara Communit...,238389,NaN,NaN,NaN,NaN,Municipal services\nOversees the activities of...,NaN


In [5]:
def remove_newlines(text):
    if isinstance(text, str):
        return text.replace('\n', ' ')
    return text

In [6]:
# Concatenate selected fields into a text corpus
open_df["corpus"] = (
    open_df["names"] + 
    " is a organisation with services of " +
    open_df["services"].fillna('').apply(remove_newlines) + 
    " " + 
    open_df["comments"].fillna('').apply(remove_newlines) + 
    " " + 
    open_df["taglines"].fillna('').apply(remove_newlines) 
     +
    open_df["audiences"].apply(lambda x: f" targeted for audiences of {x}" if isinstance(x, str) and x.strip() != '' else '')

)



In [7]:
open_df['corpus'] = open_df['corpus'].apply(remove_newlines)
open_corpus = open_df[["ids", "corpus"]].reset_index(drop=True)
open_corpus.values[4]

array([194001,
       'Adelaide Turf Cricket Association is a organisation with services of Coordination & promotion of community turf cricket in South Australia Information about local turf cricket clubs  '],
      dtype=object)

In [8]:
open_corpus

,ids,corpus
0,193932,RSL Ardrossan Sub Branch is a organisation wit...
1,193940,"RSL Brinkworth, Koolunga & Yacka Sub Branch is..."
2,193945,RSL Coonawarra-Penola Sub Branch is a organisa...
3,193975,RSL Port Pirie & Military Museum is a organisa...
4,194001,Adelaide Turf Cricket Association is a organis...
...,...,...
8461,238386,Yalata Community Council is a organisation wit...
8462,238387,Gerard Aboriginal Community Council is a organ...
8463,238388,Adelaide Hills Garden Affair is a organisation...
8464,238389,Anangu Pitjantjatjara Yankunytjatjara Communit...


In [9]:
# nlp = spacy.load(r"/Users/camilleko/NLP_summarization/src/model-best")

# def correct_grammar(text):
#     doc = nlp(text)
#     # corrected_text = "".join(token.text_with_ws for token in doc)
#     # corrected_text = corrected_text.capitalize()
#     # return corrected_text
#     cleaned_tokens = []
    
#        # Initialize a flag to capitalize the first word in each sentence
#     capitalize_next = True

#     # Iterate through the tokens in the processed text
#     for token in doc:
#         # Remove leading and trailing whitespace
#         cleaned_token = token.text.strip()
        
#         # Check if the token starts a new sentence and capitalize it
#         if capitalize_next:
#             cleaned_token = cleaned_token.capitalize()
#             capitalize_next = False
        
#         # Check if the token ends a sentence and set the flag to capitalize the next word
#         if cleaned_token.endswith(('.', '!', '?')):
#             capitalize_next = True
        
#         # Append the cleaned token to the list
#         cleaned_tokens.append(cleaned_token)

#     # Join the cleaned tokens with spaces and add a period at the end
#     cleaned_text = " ".join(cleaned_tokens) + '.'

#     return cleaned_text.strip()
# # Apply grammar correction to the "corpus" column
# open_df["corpus_new"] = open_df["corpus"].apply(correct_grammar)


In [10]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/camilleko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
def correct_nltk(text):
    # Tokenize the text into sentences using NLTK
    sentences = sent_tokenize(text)

    # Initialize a list to store cleaned sentences
    cleaned_sentences = []
    
    # Iterate through the sentences
    for sentence in sentences:  
    # Tokenize the sentence into words using NLTK
        words = word_tokenize(sentence)
        
        # Capitalize proper nouns (words starting with an uppercase letter)
        cleaned_words = [word.capitalize() if word[0].isupper() else word.lower() for word in words]
        
        # Join the cleaned words
        cleaned_sentence = ' '.join(cleaned_words)
        
        # Append the cleaned sentence to the list
        cleaned_sentences.append(cleaned_sentence)

    # Join the cleaned sentences into a single text
    cleaned_text = ' '.join(cleaned_sentences) 

    # Print the cleaned and capitalized text
    return cleaned_text

open_df["corpus_new"] = open_df["corpus"].apply(correct_nltk)


In [12]:
open_df[['ids','corpus','corpus_new']]
print(open_df['corpus'].values[0])
print(open_df['corpus_new'].values[0])

RSL Ardrossan Sub Branch is a organisation with services of Welfare and pensions support for ex-servicemen and their families Social and recreational activities Commemoration activities - ANZAC Day, Remembrance Day and other significant events   targeted for audiences of Serving and ex-service men and women and their dependents
Rsl Ardrossan Sub Branch is a organisation with services of Welfare and pensions support for ex-servicemen and their families Social and recreational activities Commemoration activities - Anzac Day , Remembrance Day and other significant events targeted for audiences of Serving and ex-service men and women and their dependents


In [13]:
open_df['corpus_new']

0        Rsl Ardrossan Sub Branch is a organisation wit...
7        Rsl Brinkworth , Koolunga & Yacka Sub Branch i...
11       Rsl Coonawarra-penola Sub Branch is a organisa...
36       Rsl Port Pirie & Military Museum is a organisa...
56       Adelaide Turf Cricket Association is a organis...
                               ...                        
14341    Yalata Community Council is a organisation wit...
14342    Gerard Aboriginal Community Council is a organ...
14343    Adelaide Hills Garden Affair is a organisation...
14344    Anangu Pitjantjatjara Yankunytjatjara Communit...
14346    Kazoku Judokan is a organisation with services...
Name: corpus_new, Length: 8466, dtype: object

In [14]:
closed_df = pd.read_csv('/Users/camilleko/NLP_summarization/data/interim/df_closed_ready4EDA.csv',index_col=0)
closed_df

/var/folders/zq/_tdy8s692b58nlb1wp2wn16c0000gn/T/ipykernel_64819/1255221936.py:1: DtypeWarning: Columns (35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  closed_df = pd.read_csv('/Users/camilleko/NLP_summarization/data/interim/df_closed_ready4EDA.csv',index_col=0)


,names,ids,Subject_id #1,Subject_id #2,Subject_id #3,Subject_id #4,Subject_id #5,Subject_id #6,Subject_id #7,Subject_id #8,...,Subject_id #85,Subject_id #86,clean_cities,Organisation Type,Also Known As,Former Name,Acronym,Locations_sm,Locations_lg,org_name_no_branch
0,RSL Ardrossan Sub Branch,193932,Ex-Defence Service Groups,Halls For Hire,Social & Activity Groups,Support & Resource Groups,Veterans,NaN,NaN,NaN,...,NaN,NaN,ardrossan,Community,Ardrossan RSL,NaN,NaN,[],[],RSL Sub Branch
7,"RSL Brinkworth, Koolunga & Yacka Sub Branch",193940,Ex-Defence Service Groups,Social & Activity Groups,Support & Resource Groups,Veterans,NaN,NaN,NaN,NaN,...,NaN,NaN,brinkworth,Community,Brinkworth RSL,NaN,NaN,[],[],"RSL Brinkworth , & Yacka Sub Branch"
11,RSL Coonawarra-Penola Sub Branch,193945,Social & Activity Groups,Support & Resource Groups,Veterans,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,penola,Community,Coonawarra RSL,NaN,NaN,[],[],RSL Coonawarra - Sub Branch
36,RSL Port Pirie & Military Museum,193975,Museums,Social & Activity Groups,Support & Resource Groups,Veterans,NaN,NaN,NaN,NaN,...,NaN,NaN,port pirie west,Community,Port Pirie RSL,NaN,NaN,[],[],RSL & Military Museum
56,Adelaide Turf Cricket Association,194001,Cricket,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,north adelaide,Community,"Turf Cricket Association, Adelaide",NaN,ATCA,[],[],Turf Cricket Association
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14341,Yalata Community Council,238386,Local Government,Councils,Aboriginal People,Indigenous People,NaN,NaN,NaN,NaN,...,NaN,NaN,yalata,Government,YALATA ANANGU ABORIGINAL CORPORATION,NaN,NaN,[],[],Community Council
14342,Gerard Aboriginal Community Council,238387,Local Government,Councils,Aboriginal People,Indigenous People,NaN,NaN,NaN,NaN,...,NaN,NaN,gerard,Government,GERARD COMMUNITY COUNCIL ABORIGINAL CORPORATION,NaN,NaN,[],[],Aboriginal Community Council
14343,Adelaide Hills Garden Affair,238388,Garden Appreciation,Gardening,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,aldgate,Government,NaN,NaN,NaN,['Adelaide Hills Garden Affair'],[],Garden Affair
14344,Anangu Pitjantjatjara Yankunytjatjara Communit...,238389,Local Government,Councils,Aboriginal People,Indigenous People,NaN,NaN,NaN,NaN,...,NaN,NaN,umuwa,Government,APY Lands,NaN,APY,[],[],Yankunytjatjara Community Council


In [15]:
closed_df["corpus"] = (
    closed_df["names"] + 
    " is a " +
    closed_df["Organisation Type"].fillna('').apply(remove_newlines)
)

# Initialize a list to store non-empty subjects
non_empty_subjects = []


# Iterate through rows and concatenate subjects for each row
for index, row in closed_df.iterrows():
    non_empty_subjects = [row[f"Subject_id #{i}"] for i in range(1, 87) if isinstance(row[f"Subject_id #{i}"], str)]
    if non_empty_subjects:
        # Add "and" before the last item in the list
        if len(non_empty_subjects) > 1:
            joined_subjects = ", ".join(non_empty_subjects[:-1]) + " and " + non_empty_subjects[-1]
        else:
            joined_subjects = non_empty_subjects[0]
        closed_df.at[index, "corpus"] += " " + joined_subjects

# Add a period at the end of the "corpus" column
closed_df["corpus"] += "."

# Display the resulting DataFrame with the "corpus" column
print(closed_df)

                                                   names     ids  \
0                               RSL Ardrossan Sub Branch  193932   
7            RSL Brinkworth, Koolunga & Yacka Sub Branch  193940   
11                      RSL Coonawarra-Penola Sub Branch  193945   
36                      RSL Port Pirie & Military Museum  193975   
56                     Adelaide Turf Cricket Association  194001   
...                                                  ...     ...   
14341                           Yalata Community Council  238386   
14342                Gerard Aboriginal Community Council  238387   
14343                       Adelaide Hills Garden Affair  238388   
14344  Anangu Pitjantjatjara Yankunytjatjara Communit...  238389   
14346                                     Kazoku Judokan  238393   

                   Subject_id #1              Subject_id #2  \
0      Ex-Defence Service Groups             Halls For Hire   
7      Ex-Defence Service Groups   Social & Activity Grou

In [16]:
closed_df[['ids','corpus']].values[0]

array([193932,
       'RSL Ardrossan Sub Branch is a Community Ex-Defence Service Groups, Halls For Hire, Social & Activity Groups, Support & Resource Groups and Veterans.'],
      dtype=object)

In [17]:
closed_df["closed_corpus_new"] = closed_df["corpus"].apply(correct_nltk)


In [18]:
closed_df[['ids','corpus', 'closed_corpus_new']].values[0]

array([193932,
       'RSL Ardrossan Sub Branch is a Community Ex-Defence Service Groups, Halls For Hire, Social & Activity Groups, Support & Resource Groups and Veterans.',
       'Rsl Ardrossan Sub Branch is a Community Ex-defence Service Groups , Halls For Hire , Social & Activity Groups , Support & Resource Groups and Veterans .'],
      dtype=object)

In [19]:
combine_df = pd.merge(open_df[['ids','corpus_new']], closed_df[['ids','closed_corpus_new']], on='ids', how='inner')

# Reset the index of the combined DataFrame
combine_df.reset_index(drop=True, inplace=True)


In [20]:
combine_df['corpus_new']

0       Rsl Ardrossan Sub Branch is a organisation wit...
1       Rsl Brinkworth , Koolunga & Yacka Sub Branch i...
2       Rsl Coonawarra-penola Sub Branch is a organisa...
3       Rsl Port Pirie & Military Museum is a organisa...
4       Adelaide Turf Cricket Association is a organis...
                              ...                        
8461    Yalata Community Council is a organisation wit...
8462    Gerard Aboriginal Community Council is a organ...
8463    Adelaide Hills Garden Affair is a organisation...
8464    Anangu Pitjantjatjara Yankunytjatjara Communit...
8465    Kazoku Judokan is a organisation with services...
Name: corpus_new, Length: 8466, dtype: object

In [21]:
combine_df['combined_corpus_new'] = combine_df['corpus_new'] + '. ' + combine_df['closed_corpus_new']

In [22]:
combine_df

,ids,corpus_new,closed_corpus_new,combined_corpus_new
0,193932,Rsl Ardrossan Sub Branch is a organisation wit...,Rsl Ardrossan Sub Branch is a Community Ex-def...,Rsl Ardrossan Sub Branch is a organisation wit...
1,193940,"Rsl Brinkworth , Koolunga & Yacka Sub Branch i...","Rsl Brinkworth , Koolunga & Yacka Sub Branch i...","Rsl Brinkworth , Koolunga & Yacka Sub Branch i..."
2,193945,Rsl Coonawarra-penola Sub Branch is a organisa...,Rsl Coonawarra-penola Sub Branch is a Communit...,Rsl Coonawarra-penola Sub Branch is a organisa...
3,193975,Rsl Port Pirie & Military Museum is a organisa...,Rsl Port Pirie & Military Museum is a Communit...,Rsl Port Pirie & Military Museum is a organisa...
4,194001,Adelaide Turf Cricket Association is a organis...,Adelaide Turf Cricket Association is a Communi...,Adelaide Turf Cricket Association is a organis...
...,...,...,...,...
8461,238386,Yalata Community Council is a organisation wit...,Yalata Community Council is a Government Local...,Yalata Community Council is a organisation wit...
8462,238387,Gerard Aboriginal Community Council is a organ...,Gerard Aboriginal Community Council is a Gover...,Gerard Aboriginal Community Council is a organ...
8463,238388,Adelaide Hills Garden Affair is a organisation...,Adelaide Hills Garden Affair is a Government G...,Adelaide Hills Garden Affair is a organisation...
8464,238389,Anangu Pitjantjatjara Yankunytjatjara Communit...,Anangu Pitjantjatjara Yankunytjatjara Communit...,Anangu Pitjantjatjara Yankunytjatjara Communit...


In [23]:
combine_df['combined_corpus_new']

0       Rsl Ardrossan Sub Branch is a organisation wit...
1       Rsl Brinkworth , Koolunga & Yacka Sub Branch i...
2       Rsl Coonawarra-penola Sub Branch is a organisa...
3       Rsl Port Pirie & Military Museum is a organisa...
4       Adelaide Turf Cricket Association is a organis...
                              ...                        
8461    Yalata Community Council is a organisation wit...
8462    Gerard Aboriginal Community Council is a organ...
8463    Adelaide Hills Garden Affair is a organisation...
8464    Anangu Pitjantjatjara Yankunytjatjara Communit...
8465    Kazoku Judokan is a organisation with services...
Name: combined_corpus_new, Length: 8466, dtype: object

In [24]:
sampled_entities = combine_df['combined_corpus_new'].sample(n=2034, random_state=42)  # Use a specific random_state for reproducibility


In [25]:
exBERT_raw_text = ' '.join(combine_df['combined_corpus_new'].values)
exBERT_raw_text = exBERT_raw_text.replace("& nbsp ;", " ").replace("& gt ;", " ")
exBERT_raw_text = exBERT_raw_text.replace("..", " ")

exBERT_raw_text_open = ' '.join(open_df['corpus_new'].values)
exBERT_raw_text_open = exBERT_raw_text_open.replace("& nbsp ;", " ").replace("& gt ;", " ")
exBERT_raw_text_open = exBERT_raw_text_open.replace("..", " ")



In [26]:
sampled_exBERT_raw_text = ' '.join(sampled_entities.values)
sampled_exBERT_raw_text = sampled_exBERT_raw_text.replace("& nbsp ;", " ").replace("& gt ;", " ")
sampled_exBERT_raw_text = sampled_exBERT_raw_text.replace("..", " ")


In [27]:
print(exBERT_raw_text[:500])

Rsl Ardrossan Sub Branch is a organisation with services of Welfare and pensions support for ex-servicemen and their families Social and recreational activities Commemoration activities - Anzac Day , Remembrance Day and other significant events targeted for audiences of Serving and ex-service men and women and their dependents. Rsl Ardrossan Sub Branch is a Community Ex-defence Service Groups , Halls For Hire , Social & Activity Groups , Support & Resource Groups and Veterans . Rsl Brinkworth , 


In [28]:
print(sampled_exBERT_raw_text[:500])

Whyalla Model Aero Sports Inc. is a organisation with services of Model Aircraft Whyalla Model Aero Sports aims to develop and foster the interests of model aviation to instruct persons in flying model aircraft.   Mostly fixed wing , however helicopters and quadcopters welcome . Model Aircraft Flying. Whyalla Model Aero Sports Inc. is a Government Model Flying . Glanville Par 3 Golf Course is a organisation with services of Nine hole par 3 golf course Barbecue facilities Club hire. Glanville Par


In [29]:
# Specify the file path where you want to save the text
file_path = "/Users/camilleko/NLP_summarization/data/interim/custom_bert_raw_text_open.txt"

# Open the file in write mode and write the concatenated text
with open(file_path, "w", encoding="utf-8") as file:
    file.write(exBERT_raw_text_open)

# Confirm that the text has been written to the file
print(f"The concatenated text has been written to {file_path}")

The concatenated text has been written to /Users/camilleko/NLP_summarization/data/interim/custom_bert_raw_text_open.txt


In [30]:
# # Specify the file path where you want to save the text
# file_path = "/Users/camilleko/NLP_summarization/data/interim/custom_bert_raw_text_open.txt"

# # Open the file in write mode and write the concatenated text
# with open(file_path, "w", encoding="utf-8") as file:
#     file.write(exBERT_raw_text)

# # Confirm that the text has been written to the file
# print(f"The concatenated text has been written to {file_path}")

In [43]:
# Specify the file path where you want to save the text
file_path = "/Users/camilleko/NLP_summarization/data/interim/sampled_exBERT_raw_text.txt"

# Open the file in write mode and write the concatenated text
with open(file_path, "w", encoding="utf-8") as file:
    file.write(sampled_exBERT_raw_text)

# Confirm that the text has been written to the file
print(f"The concatenated text has been written to {file_path}")

The concatenated text has been written to /Users/camilleko/NLP_summarization/data/interim/sampled_exBERT_raw_text.txt


In [41]:
filter_id = open_df['ids'].to_list()
print(len(filter_id))

8466


In [61]:
import pandas as pd

# Define the path to the CSV file
csv_file_path = "/Users/camilleko/NLP_summarization/data/interim/GT_format.csv"

# Use pandas to read the CSV file into a DataFrame
df_gt = pd.read_csv(csv_file_path, sep=',', encoding='utf-8')


type(df_gt['Internal Contact ID'][0])
df_gt

,Organization Name,Internal Contact ID,Comment,GT_summary,Unnamed: 4
0,RSL Ardrossan Sub Branch,193932,NaN,The RSL was founded in 1916 to provide comrade...,NaN
1,RSL Balaklava Sub Branch,193933,NaN,The RSL was founded in 1916 to provide comrade...,NaN
2,RSL Barmera Sub Branch,193934,NaN,The RSL was founded in 1916 to provide comrade...,NaN
3,RSL Berri Sub Branch,193935,NaN,The RSL was founded in 1916 to provide comrade...,NaN
4,RSL Blanchetown Sub Branch,193936,NaN,The RSL was founded in 1916 to provide comrade...,NaN
...,...,...,...,...,...
14402,Lutheran Church - Karoonda,238400,&nbsp;Lutheran Churches services,NaN,NaN
14403,Uniting Church - Karoonda,238401,Uniting Church services,NaN,NaN
14404,Avon District Hall Inc.,238407,NaN,NaN,NaN
14405,Karoonda Men's Shed,238408,Social and workshop activities for men,NaN,NaN


In [75]:
highlight = {}
for id in filter_id:
    if id in df_gt['Internal Contact ID'].values:
        highlight[id] = df_gt.loc[df_gt['Internal Contact ID'] == id, 'GT_summary'].values[0]


In [76]:
# Filter out key-value pairs with non-nan values
filtered_highlight = {key: value for key, value in highlight.items() if not pd.isna(value)}

# Print the filtered dictionary
print(filtered_highlight) #####add @highlight\n\n to each values

{193932: 'The RSL was founded in 1916 to provide comradeship and support to Australia&rsquo;s veterans and their families.&nbsp;That core mission has never changed but has continued to evolve to meet the needs of each generation of servicemen and women.&nbsp;We have a branch network that covers Australia and any veteran who needs help will get it.&nbsp; Every serving ADF member and veteran will be warmly welcomed at their local RSL.', 193940: 'The RSL was founded in 1916 to provide comradeship and support to Australia&rsquo;s veterans and their families.&nbsp;That core mission has never changed but has continued to evolve to meet the needs of each generation of servicemen and women.&nbsp;We have a branch network that covers Australia and any veteran who needs help will get it.&nbsp; Every serving ADF member and veteran will be warmly welcomed at their local RSL.', 193945: 'The RSL was founded in 1916 to provide comradeship and support to Australia&rsquo;s veterans and their families.&n

In [77]:
filtered_highlight = {key: '@highlight\n\n ' + value if not pd.isna(value) else None for key, value in highlight.items()}

# Print the filtered dictionary
print(filtered_highlight)

{193932: '@highlight The RSL was founded in 1916 to provide comradeship and support to Australia&rsquo;s veterans and their families.&nbsp;That core mission has never changed but has continued to evolve to meet the needs of each generation of servicemen and women.&nbsp;We have a branch network that covers Australia and any veteran who needs help will get it.&nbsp; Every serving ADF member and veteran will be warmly welcomed at their local RSL.', 193940: '@highlight The RSL was founded in 1916 to provide comradeship and support to Australia&rsquo;s veterans and their families.&nbsp;That core mission has never changed but has continued to evolve to meet the needs of each generation of servicemen and women.&nbsp;We have a branch network that covers Australia and any veteran who needs help will get it.&nbsp; Every serving ADF member and veteran will be warmly welcomed at their local RSL.', 193945: '@highlight The RSL was founded in 1916 to provide comradeship and support to Australia&rsquo

In [73]:
for key in filtered_highlight.keys():
    if key in open_corpus['ids']:
        filtered_highlight[key] = open_corpus['corpus'][key]

In [74]:
filtered_highlight

{193932: 'The RSL was founded in 1916 to provide comradeship and support to Australia&rsquo;s veterans and their families.&nbsp;That core mission has never changed but has continued to evolve to meet the needs of each generation of servicemen and women.&nbsp;We have a branch network that covers Australia and any veteran who needs help will get it.&nbsp; Every serving ADF member and veteran will be warmly welcomed at their local RSL.',
 193940: 'The RSL was founded in 1916 to provide comradeship and support to Australia&rsquo;s veterans and their families.&nbsp;That core mission has never changed but has continued to evolve to meet the needs of each generation of servicemen and women.&nbsp;We have a branch network that covers Australia and any veteran who needs help will get it.&nbsp; Every serving ADF member and veteran will be warmly welcomed at their local RSL.',
 193945: 'The RSL was founded in 1916 to provide comradeship and support to Australia&rsquo;s veterans and their families.